In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from Yelp_scraper import yelp_search_page
from Yelp_scraper import yelp_store_page
from tqdm import tqdm
import time
import csv
import json

def init_driver(driver_path):
    '''
    Get Selenium driver
    Parameters: None
    Return: Selenium webdriver element
    '''
    opts = Options()

    opts.add_argument("user_agent = Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.20 Safari/537.36")
    opts.add_argument("--disable-notifications")
    #opts.add_argument('--headless')
    #opts.add_argument("--window-size=720,1080")
    opts.add_argument("--lang=en")
    opts.add_argument('--no-sandbox')
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument('disable_infobars')
    drive = webdriver.Chrome(driver_path, options=opts)
        
    return drive


In [2]:
drive_path = 'C:/Users/ljw15/chromedrive99/chromedriver'  #########input the path for chrome driver
driver = init_driver(drive_path)

## 1. Choose the store name and store address want to search

In [10]:
#######initialize serach page
search_page = yelp_search_page(driver)  ########used for yelps earch page

In [11]:
store_name = 'addison'
store_address = 'La Jolla'
search_page.search(store_name,store_address)

print(search_page.has_no_result()) #see whether there are result return  
                                   # True: No result return
                                   # False: There are result

False


### a.Call get_store_list() to get all store information under this search term

In [12]:
store_list = search_page.get_store_list()

In [13]:
print(len(store_list))

27


### b.get_store_list() return a list of store information with basic features:
##### link, name, address, attribute

In [14]:
#########these are information return from search page like https://www.yelp.com/search?find_desc=addison&find_loc=San+Diego%2C+CA+92101 
#########for each store
store_list[20]

{'link': 'https://www.yelp.com/biz/coastal-moving-san-diego-15?osq=addison',
 'name': 'Coastal Moving',
 'address': 'San Diego',
 'attribute': 'Movers\nPacking Services\nSelf Storage\nSan Diego'}

## 2.Scrape detail information from one store page

In [3]:
######### create store page object
store_page = yelp_store_page(driver)    ########used for yelp individual store page

### a. Scrape store detail info

In [5]:
link = 'https://www.yelp.com/biz/coastal-moving-san-diego-15?osq=addison'
######### get store detail information
detail_data = store_page.get_store_info(link)

price Message: no such element: Unable to locate element: {"method":"css selector","selector":"span[class ^= ' css-1ir4e44']"}
  (Session info: chrome=100.0.4896.75)

['https://maps.googleapis.com/maps/api/staticmap?size=315x150', 'sensor=false', 'client=gme-yelp', 'language=en', 'scale=1', 'path=color%3A0x0947a766%7Cweight%3A0%7Cfillcolor%3A0x0947a766%7C33.116850%2C-117.433548%7C32.617400%2C-117.358704%7C32.558389%2C-117.115631%7C32.558389%2C-116.911011%7C32.731841%2C-116.911011%7C32.963739%2C-116.949463%7C33.289212%2C-117.228928%7C33.289212%2C-117.433548%7C33.116850%2C-117.433548', 'markers=scale%3A1%7Cicon%3Ahttps%3A%2F%2Fyelp-images.s3.amazonaws.com%2Fassets%2Fmap-markers%2Fannotation_32x43.png%7C32.948824%2C-117.240022', 'signature=_yVej7g_3S2CrClV6Y2jZxnT5q8=']
false
['false']
geo list index out of range


In [5]:
detail_data

{'link': 'https://www.yelp.com/biz/coastal-moving-san-diego-15?osq=addison',
 'name': 'Coastal Moving',
 'rate': '3 star rating',
 'num_reviewer': '9 reviews',
 'price': '',
 'attribute': ['Movers', 'Packing Services', 'Self Storage'],
 'is_claimed': True,
 'coordinate': {},
 'address': '12707 High Bluff Dr,Ste 200,San Diego, CA 92130'}

### b. Scrape comments detail info

In [6]:
###############get all comments under this store
comments_list = store_page.get_comments_list()

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[class ^= 'next-link']"}
  (Session info: chrome=100.0.4896.60)

Reach final page


In [7]:
comments_list[100]

{'user_name': 'Nicolas C.',
 'user_url': 'https://www.yelp.com/user_details?userid=ybOICKNLA5tlG7ZIwu_Ggw',
 'user_location': 'Grantville, San Diego, CA',
 'text': 'This place is amazing the burgers are so worth it!! I will definitely be coming back here!',
 'rate': '5',
 'time': '1/7/2022',
 'funny': 0,
 'useful': 0,
 'cool': 0}